In [2]:
#朴素贝叶斯先算 先验p(x,y)，再算后验p(Y|X)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter #计数
import math

In [7]:
#数据
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data,columns = iris.feature_names)
    df['label'] = iris.target
    df.columns = [
        'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
    ]
    data = np.array(df.iloc[:100,:])
#     print(data)
    return data[:,:-1],data[:,-1]

In [8]:
X,y = create_data()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size = 0.3)

In [24]:
labels = list(set(y))
data = {label: [] for label in labels}
for f , label in zip(X,y):
            data[label].append(f)
data

{0.0: [array([5.1, 3.5, 1.4, 0.2]),
  array([4.9, 3. , 1.4, 0.2]),
  array([4.7, 3.2, 1.3, 0.2]),
  array([4.6, 3.1, 1.5, 0.2]),
  array([5. , 3.6, 1.4, 0.2]),
  array([5.4, 3.9, 1.7, 0.4]),
  array([4.6, 3.4, 1.4, 0.3]),
  array([5. , 3.4, 1.5, 0.2]),
  array([4.4, 2.9, 1.4, 0.2]),
  array([4.9, 3.1, 1.5, 0.1]),
  array([5.4, 3.7, 1.5, 0.2]),
  array([4.8, 3.4, 1.6, 0.2]),
  array([4.8, 3. , 1.4, 0.1]),
  array([4.3, 3. , 1.1, 0.1]),
  array([5.8, 4. , 1.2, 0.2]),
  array([5.7, 4.4, 1.5, 0.4]),
  array([5.4, 3.9, 1.3, 0.4]),
  array([5.1, 3.5, 1.4, 0.3]),
  array([5.7, 3.8, 1.7, 0.3]),
  array([5.1, 3.8, 1.5, 0.3]),
  array([5.4, 3.4, 1.7, 0.2]),
  array([5.1, 3.7, 1.5, 0.4]),
  array([4.6, 3.6, 1. , 0.2]),
  array([5.1, 3.3, 1.7, 0.5]),
  array([4.8, 3.4, 1.9, 0.2]),
  array([5. , 3. , 1.6, 0.2]),
  array([5. , 3.4, 1.6, 0.4]),
  array([5.2, 3.5, 1.5, 0.2]),
  array([5.2, 3.4, 1.4, 0.2]),
  array([4.7, 3.2, 1.6, 0.2]),
  array([4.8, 3.1, 1.6, 0.2]),
  array([5.4, 3.4, 1.5, 0.4]),
  a

In [48]:
#采用高斯朴素贝叶斯GaussianNB
class NaiveBayes:
    def __init__(self):
        self.model = None
    
    #数学期望
    @staticmethod
    def mean(X):
        return sum(X)/float(len(X))
    
    #标准差、均方差，公式有点问题
    def stdev(self,X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x-avg,2) for x in X])/ (float(len(X))-1))
    
    #概率密度函数-->高斯
    def gaussian_probability(self,x,mean,stdev):
        exponent = math.exp(-(math.pow(x-mean, 2) / 
                             (2 * math.pow(stdev,2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
    
    #处理X_train
    #zip:参数中的多个迭代器取元素组合成一个新的迭代器;
    def summarize(self , train_data):
        summaries = [(self.mean(i),self.stdev(i)) for i in zip(*train_data)]
        return summaries
        
    #分类别求出数学期望和标准差
    def fit(self,X,y):
        labels = list(set(y))
        data = {label: [] for label in labels} #{0.0: [], 1.0: []}
        for f , label in zip(X,y):
            data[label].append(f) #按类别分类，类别为0的每个样本为一起，dict类型，键值对 0：[[],[]]
        self.model = {
            label: self.summarize(value)
            for label , value in data.items()#label:0/1  value:每个样本的特征值，算完后就变成期望和均方差
        }
        return 'gaussianNB train done'
    
    #计算概率,计算每个类别的高斯概率
    #dict.items() 函数以列表返回可遍历的(键, 值) 元组数组
    def calculate_probabilities(self,input_data):
        probabilities = {}
        for label , value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)): #样本数
                mean,stdev = value[i]
                probabilities[label] *= self.gaussian_probability(
                input_data[i],mean,stdev)
        return probabilities
    
    #类别
    #lambda作为一个表达式，定义了一个匿名函数，上例的代码x为入口参数，x+1为函数体
    def predict(self,X_test):
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key = lambda x:x[-1])[-1][0] #x[-1][0]，排序完后，选最后一行第一列
        return label 
    
    #得分
    def score(self,X_test,y_test):
        right = 0
        for X , y in zip(X_test,y_test):
            label = self.predict(X)
            if label == y:
                right += 1
        return right / float(len(X_test))

In [49]:
model = NaiveBayes()

In [50]:
model.fit(X_train,y_train)

'gaussianNB train done'

In [51]:
print(model.predict([4.4 , 3.2, 1.3 ,0.2]))

0.0
0.0


In [52]:
model.score(X_test,y_test)

1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0


1.0

In [43]:
#使用sklearn实例
from sklearn.naive_bayes import GaussianNB

In [44]:
clf = GaussianNB()
clf.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [45]:
clf.score(X_test,y_test)

1.0

In [46]:
clf.predict([[4.4,3.2,1.3,0.2]])

array([0.])

In [47]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB # 伯努利模型和多项式模型